#1. Import Necessary Libraries

In [19]:
import gym
import numpy as np
import random
import time
import pandas as pd
from IPython import display
from tqdm.notebook import tqdm

#2. Support Function

In [20]:
# Hyperparameters
gamma = 0.99
learning_rate = 0.1
max_epsilon = 1.0
min_epsilon = 0.01
epsilon_decay_rate = 0.005

num_episodes = 20000
num_steps_per_episode = 100

In [21]:
def play(env, q_table, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = np.argmax(q_table[state, :])
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [22]:
def play_multiple_times(env, q_table, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, q_table)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')
    return success/max_episodes 

In [23]:
runtime_data = []
winrate_data = []

#3. Q-Learning

In [24]:
def q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []
    for episode in tqdm(range(num_episodes), desc='Q-Learning...'):
        state = env.reset()

        reward_episode = 0.0
        done = False
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)
        for step in range(num_steps_per_episode):
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            next_state, reward, done, info = env.step(action)
            q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + gamma * np.max(q_table[next_state,:]))

            reward_episode += reward
            state = next_state

            if done:
                break
        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    return q_table, rewards_all

In [25]:
runtimes = []
winrates = []

##3.1. FrozeLake-v0

In [26]:
env = gym.make('FrozenLake-v0')
start = time.time()
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, 
                                  learning_rate, gamma, max_epsilon, 
                                  min_epsilon, epsilon_decay_rate)
end = time.time()
winrate = play_multiple_times(env, q_table, 1000)
runtimes.append(end-start)
winrates.append(winrate)

Q-Learning...:   0%|          | 0/20000 [00:00<?, ?it/s]

Episode 19999 finished
Number of successes: 739/1000
Average number of steps: 37.01082543978349


##3.2. FrozenLake8x8-v0

In [27]:
env = gym.make('FrozenLake8x8-v0')
start = time.time()
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, 
                                  learning_rate, gamma, max_epsilon, 
                                  min_epsilon, epsilon_decay_rate)
end = time.time()
winrate = play_multiple_times(env, q_table, 1000)
runtimes.append(end-start)
winrates.append(winrate)

Q-Learning...:   0%|          | 0/20000 [00:00<?, ?it/s]

Episode 19999 finished
Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


##3.3. Taxi-v3

In [28]:
env = gym.make('Taxi-v3')
start = time.time()
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, 
                                  learning_rate, gamma, max_epsilon, 
                                  min_epsilon, epsilon_decay_rate)
end = time.time()
winrate = play_multiple_times(env, q_table, 1000)
runtimes.append(end-start)
winrates.append(winrate)

Q-Learning...:   0%|          | 0/20000 [00:00<?, ?it/s]

Episode 19999 finished
Number of successes: 1000/1000
Average number of steps: 13.044


In [29]:
runtime_data.append(runtimes)
winrate_data.append(winrates)

#4. SARSA

In [30]:
def sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []
    for episode in tqdm(range(num_episodes), desc='SARSA...'):
        state = env.reset()
        reward_episode = 0.0
        done = False
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)
        exploration = random.uniform(0,1)
        if exploration < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state, :])

        for step in range(num_steps_per_episode):
            next_state, reward, done, info = env.step(action)
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                next_action = env.action_space.sample()
            else:
                next_action = np.argmax(q_table[next_state, :])
            q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + gamma * q_table[next_state,next_action])
            reward_episode += reward
            action = next_action
            state = next_state

            if done:
                break

        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    return q_table, rewards_all

In [31]:
runtimes = []
winrates = []

##4.1. FrozenLake-v0

In [32]:
env = gym.make('FrozenLake-v0')
start = time.time()
q_table, rewards_all = sarsa(env, num_episodes, num_steps_per_episode, 
                                  learning_rate, gamma, max_epsilon, 
                                  min_epsilon, epsilon_decay_rate)
end = time.time()
winrate = play_multiple_times(env, q_table, 1000)
runtimes.append(end-start)
winrates.append(winrate)

SARSA...:   0%|          | 0/20000 [00:00<?, ?it/s]

Episode 19999 finished
Number of successes: 740/1000
Average number of steps: 37.35675675675676


##4.2. FrozenLake8x8-v0

In [33]:
env = gym.make('FrozenLake8x8-v0')
start = time.time()
q_table, rewards_all = sarsa(env, num_episodes, num_steps_per_episode, 
                                  learning_rate, gamma, max_epsilon, 
                                  min_epsilon, epsilon_decay_rate)
end = time.time()
winrate = play_multiple_times(env, q_table, 1000)
runtimes.append(end-start)
winrates.append(winrate)

SARSA...:   0%|          | 0/20000 [00:00<?, ?it/s]

Episode 19999 finished
Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


##4.3. Taxi-v3

In [34]:
env = gym.make('Taxi-v3')
start = time.time()
q_table, rewards_all = sarsa(env, num_episodes, num_steps_per_episode, 
                                  learning_rate, gamma, max_epsilon, 
                                  min_epsilon, epsilon_decay_rate)
end = time.time()
winrate = play_multiple_times(env, q_table, 1000)
runtimes.append(end-start)
winrates.append(winrate)

SARSA...:   0%|          | 0/20000 [00:00<?, ?it/s]

Episode 19999 finished
Number of successes: 1000/1000
Average number of steps: 13.13


In [35]:
runtime_data.append(runtimes)
winrate_data.append(winrates)

#5. Conclusion

In [36]:
runtime_df = pd.DataFrame(runtime_data, columns=['FrozenLake-v0', 'FrozenLake8x8-v0', 'taxi-v3'], 
                          index=['Q-Learning', 'SARSA'])
winrate_df = pd.DataFrame(winrate_data, columns=['FrozenLake-v0', 'FrozenLake8x8-v0', 'taxi-v3'], 
                          index=['Q-Learning', 'SARSA'])
runtime_df.columns = pd.MultiIndex.from_product([['RUNTIME'], runtime_df.columns.tolist()])
winrate_df.columns = pd.MultiIndex.from_product([['WINRATE'], winrate_df.columns.tolist()])
display.display(runtime_df)
display.display(winrate_df)

RUNTIME                            
           FrozenLake-v0 FrozenLake8x8-v0    taxi-v3
Q-Learning     25.115748        60.138200  12.805272
SARSA          14.147506        35.223214   6.996474

WINRATE                         
           FrozenLake-v0 FrozenLake8x8-v0 taxi-v3
Q-Learning         0.739              0.0     1.0
SARSA              0.740              0.0     1.0

In [38]:
#   Nhìn chung cả hai thuật toán đều đảm bảo hội tụ về một phương án tối ưu, tuy nhiên vì game FronzenLake8x8 
# quá khó nên cả 2 đều không thể thắng được 
#   Trong cả 3 games, thời gian thực thi của SARSA luôn nhanh hơn Q-Learning (nhanh gấp đôi Q-Learning)